# Ch11c_Tracking_BERT_Fine-tuning_with_W&B 

In [1]:
import torch

In [2]:
!pip install transformers datasets

     |████████████████████████████████| 2.5MB 9.0MB/s 
     |████████████████████████████████| 266kB 57.0MB/s 
     |████████████████████████████████| 901kB 44.8MB/s 
     |████████████████████████████████| 3.3MB 40.6MB/s 
     |████████████████████████████████| 122kB 72.5MB/s 
     |████████████████████████████████| 245kB 60.9MB/s 


In [3]:
from transformers.trainer_utils import set_seed
set_seed(42)

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
!pip install wandb

     |████████████████████████████████| 1.8MB 8.7MB/s 
     |████████████████████████████████| 143kB 64.9MB/s 
     |████████████████████████████████| 102kB 16.7MB/s 
     |████████████████████████████████| 174kB 60.4MB/s 
     |████████████████████████████████| 71kB 12.2MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=79fe1d6d571a19484c1be7de8de55df58159b37216d652adedba2e4ac7ebdfe5
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=69e3c27100fcf04d92b77fb0db144d84b6e1e925a4b7fb907e8e9c79fcde76af
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [6]:
import wandb
# or
# export WANDB_API_KEY= PUT-YOUR-API-KEY-HERE
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Loading pre-trained model and tokenizer

In [7]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
model_path= 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path, id2label={0:"NEG", 1:"POS"}, label2id={"NEG":0, "POS":1})

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

## Loading popular IMDB dataset

In [8]:
from datasets import load_dataset

# to take entire dataset from original train 25 K AND TEST 25K
'''
imdb_train= load_dataset('imdb', split="train")
imdb_test= load_dataset('imdb', split="test[:6250]+test[-6250:]")
imdb_val= load_dataset('imdb', split="test[6250:12500]+test[-12500:-6250]")

'''
# to take smaller portion 4K for train, 1K for test and 1K for validation
imdb_train= load_dataset('imdb', split="train[:2000]+train[-2000:]")
imdb_test= load_dataset('imdb', split="test[:500]+test[-500:]")
imdb_val= load_dataset('imdb', split="test[500:1000]+test[-1000:-500]")


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [12]:
imdb_train.shape, imdb_test.shape, imdb_val.shape

((4000, 2), (1000, 2), (1000, 2))

In [13]:
enc_train = imdb_train.map(lambda e: tokenizer( e['text'], padding=True, truncation=True), batched=True, batch_size=1000) 
enc_test =  imdb_test.map(lambda e: tokenizer( e['text'], padding=True, truncation=True), batched=True, batch_size=1000) 
enc_val =   imdb_val.map(lambda e: tokenizer( e['text'], padding=True, truncation=True), batched=True, batch_size=1000) 

Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-78dbc0df9bbc2138.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-9110b218d4f98e9a.arrow


In [14]:
import pandas as pd
pd.DataFrame(enc_train)

,attention_mask,input_ids,label,text
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 22953, 2213, 4381, 2152, 2003, 1037, 947...",1,Bromwell High is a cartoon comedy. It ran at t...
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 11573, 2791, 1006, 2030, 2160, 24913, 20...",1,Homelessness (or Houselessness as George Carli...
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 8235, 2058, 1011, 3772, 2011, 23920, 575...",1,Brilliant over-acting by Lesley Ann Warren. Be...
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2023, 2003, 4089, 1996, 2087, 2104, 9250...",1,This is easily the most underrated film inn th...
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2023, 2003, 2025, 1996, 5171, 11463, 837...",1,This is not the typical Mel Brooks film. It wa...
...,...,...,...,...
3995,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2875, 1996, 2203, 1997, 1996, 3185, 1010...",0,"Towards the end of the movie, I felt it was to..."
3996,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2023, 2003, 1996, 2785, 1997, 3185, 2008...",0,This is the kind of movie that my enemies cont...
3997,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2387, 1005, 6934, 1005, 2197, 2305...",0,I saw 'Descent' last night at the Stockholm Fi...
3998,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2070, 3152, 2008, 2017, 4060, 2039, 2005...",0,Some films that you pick up for a pound turn o...


## Preparing training settings with TrainingArguments and Trainer class

TrainingArguments is the subset of the arguments we use in our example scripts **which relate to the training loop
itself**.

In [15]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./MyIMDBModel', 
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0 
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,                
    weight_decay=0.01,
    logging_strategy='steps', 
    # TensorBoard log directory               
    logging_dir='./logs',            
    logging_steps=50,
    # other options : no, steps
    evaluation_strategy="steps",
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True,
    learning_rate=5e-5,
    report_to="wandb",
    run_name="IMDB-batch-16-lr-5e-5"
    )

Let's design our evaluation metric as follows:

In [16]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [17]:
# for a fresh copy of distilBert
# model = DistilBertForSequenceClassification.from_pretrained(model_path, id2label={0:"NEG", 1:"POS"}, label2id={"NEG":0, "POS":1})

trainer = Trainer(
    # the pre-trained model that will be fine-tuned 
    model=model,
     # training arguments that we defined above                        
    args=training_args,
    # training and validation dataset                 
    train_dataset=enc_train,         
    eval_dataset=enc_val,            
    compute_metrics= compute_metrics
)

Using amp fp16 backend


Trainer class is an optimized tool for Transformer to organize training and evalulation process for PyTorch and Tensorflow.

In [18]:
results=trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 4000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 750
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: savasy (use `wandb login --relogin` to force relogin)


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.681400,0.580819,0.800000,0.799769,0.801394,0.800000
100,0.419900,0.453349,0.806000,0.802870,0.826750,0.806000
150,0.355900,0.354100,0.855000,0.853945,0.865565,0.855000
200,0.318800,0.276682,0.902000,0.901975,0.902412,0.902000
250,0.282600,0.265800,0.899000,0.898955,0.899705,0.899000
300,0.196000,0.337965,0.889000,0.888711,0.893090,0.889000
350,0.228000,0.289109,0.906000,0.905994,0.906104,0.906000
400,0.168800,0.300406,0.907000,0.906932,0.908190,0.907000
450,0.168600,0.455914,0.871000,0.869825,0.884895,0.871000
500,0.154700,0.336408,0.898000,0.897882,0.899849,0.898000


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Saving model checkpoint to ./MyIMDBModel/checkpoint-50
Configuration saved in ./MyIMDBModel/checkpoint-50/config.json
Model weights saved in ./MyIMDBModel/checkpoint-50/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation

Finishing WandB

In [19]:
wandb.finish()

train/loss,0.0568
train/learning_rate,0.0
train/epoch,3.0
train/global_step,750
_runtime,524
_timestamp,1626250289
_step,30
eval/loss,0.37165
eval/Accuracy,0.908
eval/F1,0.90799
eval/Precision,0.90816


train/loss,█▅▄▄▄▃▃▂▂▂▁▁▁▁▁
train/learning_rate,▅██▇▇▆▅▅▄▄▃▃▂▂▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
eval/loss,█▅▃▁▁▃▂▂▅▃▄▃▄▃▃
eval/Accuracy,▁▁▄▇▇▇██▅▇▇█▇██
eval/F1,▁▁▄▇▇▇██▅▇▇█▇██
eval/Precision,▁▃▅▇▇▇██▆▇▇█▇██
